In [ ]:
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
SHAPEFILE_PATH = "ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp"
OUTPUT_PATH = "mapa_clusters_iniciais.png"

In [ ]:
BASE_DIR = os.getcwd()
BASE_DIR = os.path.dirname(BASE_DIR)  # sobe 1 nível
csv_path = os.path.join(BASE_DIR, "dados", "coordenadas_consultor.csv")
df_consultor = pd.read_csv(csv_path, encoding="latin1", delimiter=",", low_memory=False)
csv_path = os.path.join(BASE_DIR, "dados", "escolas_atribuidas_v0.csv")
df_escolas_com_cluster = pd.read_csv(csv_path, encoding="latin1", delimiter=";", low_memory=False)

In [ ]:
def add_receitas_consultor(df_assigned: pd.DataFrame, df_consultor: pd.DataFrame) -> tuple[pd.DataFrame, float]:

    df_consultor_out = df_consultor.copy()

    # Agrupa as escolas por cluster (índice do consultor) e soma seus valores 'AB'.
    receita_do_cluster = df_assigned.groupby('cluster')['AB'].sum()

    df_consultor_out = df_consultor_out.join(receita_do_cluster.rename('receita'))
    df_consultor_out['receita'] = df_consultor_out['receita'].fillna(0)

    # Calcula a receita provável média (x).
    AB = df_assigned['AB'].sum()
    x = AB / max(1, len(df_consultor_out))
    df_consultor_out['receita_minus_media'] = df_consultor_out['receita'] - x

    return df_consultor_out, x

In [ ]:
# -------------------------
# Plotting with shapefile (Brazil)
# -------------------------
def plot_clusters_on_brazil(df_schools_with_clusters: pd.DataFrame, df_bases: pd.DataFrame,
                            shapefile_path: str, output_image_path: str):
    """
    Plot clusters overlaying Brazil polygon from shapefile.
    - shapefile_path: path to countries shapefile (.shp)
    - output_image_path: full path to save PNG
    """
    # try to load shapefile and robustly find Brazil
    world = gpd.read_file(shapefile_path)
    brasil = None
    # common column names to try
    for col in ['NAME', 'name', 'NAME_LONG', 'admin', 'sovereignt']:
        if col in world.columns:
            # some shapefiles use 'Brazil' or 'Brazil (some)'
            mask = world[col].astype(str).str.contains('Brazil', case=False, na=False)
            if mask.any():
                brasil = world[mask]
                break
    if brasil is None:
        # fallback: choose by ISO_A3 if present
        if 'iso_a3' in world.columns:
            brasil = world[world['iso_a3'] == 'BRA']
    if brasil is None:
        # last fallback: largest polygon (very crude)
        brasil = world.loc[[world.geometry.area.idxmax()]]

    # explode multi-part geometries (compatível com geopandas versions)
    try:
        brasil = brasil.explode(index_parts=False).reset_index(drop=True)
    except TypeError:
        brasil = brasil.explode().reset_index(drop=True)

    # prepare GeoDataFrames
    df = df_schools_with_clusters.copy()
    df = df.rename(columns={c: c.strip().lower() for c in df.columns})
    if not {'latitude', 'longitude', 'cluster'}.issubset(set(df.columns)):
        raise ValueError("df_schools_with_clusters deve conter 'latitude','longitude','cluster'.")

    gdf_sch = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['longitude'], df['latitude']), crs="EPSG:4326")
    gdf_bases = gpd.GeoDataFrame(df_bases.copy(), geometry=gpd.points_from_xy(df_bases['longitude'], df_bases['latitude']), crs="EPSG:4326")

    # plotting
    fig, ax = plt.subplots(figsize=(10, 12))
    brasil.plot(ax=ax, color="white", edgecolor="black")
    # color map
    n_clusters = int(df['cluster'].nunique())
    # use categorical colormap with recycling if > 20
    base_cmap = plt.get_cmap('tab20')
    for idx, cluster_id in enumerate(sorted(df['cluster'].unique())):
        subset = df[df['cluster'] == cluster_id]
        color = base_cmap(idx % 20)
        ax.scatter(subset['longitude'], subset['latitude'], c=[color], s=8, alpha=0.6, label=f"c{cluster_id}")

    # plot bases
    ax.scatter(df_bases['longitude'], df_bases['latitude'], marker='*', s=140, c='k', edgecolor='white', linewidth=0.6, zorder=5)
    # annotate bases with consultor_id
    for _, row in df_bases.iterrows():
        ax.annotate(str(int(row['consultor_id'])), xy=(row['longitude'], row['latitude']),
                    xytext=(3,3), textcoords="offset points", fontsize=9)

    ax.set_title("Clusters iniciais (Voronoi) — escolas coloridas por consultor", fontsize=14)
    # small legend outside
    ax.legend(loc='lower left', bbox_to_anchor=(1.01, 0), fontsize='small', markerscale=2)
    plt.tight_layout()
    os.makedirs(os.path.dirname(output_image_path) or '.', exist_ok=True)
    plt.savefig(output_image_path, dpi=300, bbox_inches='tight')
    plt.close(fig)
    print(f"✅ Mapa salvo em: {output_image_path}")

In [ ]:
# Verificando se o shapefile existe antes de tentar plotar
if os.path.exists(SHAPEFILE_PATH):
    plot_clusters_on_brazil(
        df_schools_with_clusters=df_escolas_com_cluster, 
        df_bases=df_consultor,
        shapefile_path=SHAPEFILE_PATH, 
        output_image_path=OUTPUT_PATH
    )
else:
    print(f"⚠️ Atenção: Shapefile não encontrado em '{SHAPEFILE_PATH}'. O mapa não pôde ser gerado.")
    print("    Você pode baixar um shapefile do Brasil do IBGE ou de fontes como o Natural Earth.")

In [ ]:
df_consultor_com_receita, media_receita = add_receitas_consultor(df_escolas_com_cluster, df_consultor)

# ===================================================================
# 2. GERANDO O GRÁFICO DE BARRAS
# ===================================================================

# Ordenando o DataFrame pela receita para melhor visualização
df_plot = df_consultor_com_receita.sort_values('receita', ascending=False).copy()

# Definindo as cores: azul para acima da média, cinza para abaixo
colors = ['#4A90E2' if val >= 0 else '#B0B0B0' for val in df_plot['receita_minus_media']]

# Criando a figura e os eixos para o gráfico
fig, ax = plt.subplots(figsize=(12, 7))

# Criando as barras
bars = ax.bar(
    df_plot['consultor_id'].astype(str), 
    df_plot['receita'],
    color=colors
)

# Adicionando a linha da média
ax.axhline(
    media_receita, 
    color='#D0021B', 
    linestyle='--', 
    linewidth=2,
    label=f'Média de Receita (R$ {media_receita:,.2f})'
)

# Formatando os rótulos do eixo Y 
from matplotlib.ticker import FuncFormatter
formatter = FuncFormatter(lambda y, pos: f'R$ {int(y/1000)}k' if y > 0 else 'R$ 0')
ax.yaxis.set_major_formatter(formatter)

# Adicionando títulos e rótulos
ax.set_title('Receita por Consultor vs. Média', fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('ID do Consultor', fontsize=12)
ax.set_ylabel('Receita Atribuída', fontsize=12)
ax.legend()

# Melhorando a aparência geral
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.yaxis.grid(True, linestyle='--', alpha=0.6)
ax.tick_params(axis='x', rotation=0)

# Salvando o gráfico em um arquivo
output_path = 'grafico_receita_por_consultor.png'
plt.tight_layout()
plt.savefig(output_path, dpi=300)
plt.close(fig)